In [ ]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from keras.models import Sequential
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
warnings.simplefilter(action='ignore', category= FutureWarning)
%matplotlib inline

In [ ]:
train_path = '[Path to File]'
test_path = '[Path to File]'
batch_size = 16

datagen = ImageDataGenerator() 

train_generator = datagen.flow_from_directory(
    directory = train_path,
    batch_size = batch_size,
    target_size=(224, 224),
    color_mode= 'rgb',
    class_mode= 'categorical',
    seed = 123,
    shuffle = True,
    )
    



test_generator = datagen.flow_from_directory(
    directory = test_path,
    batch_size = batch_size,
    target_size=(224, 224),
    color_mode= 'rgb',
    class_mode= 'categorical',
    shuffle = False,
    seed = 123,
)

In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(rate=0.5)(x)
preds = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)
print(model.summary())

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer ='SGD', loss='categorical_crossentropy', metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

                      

In [ ]:
history = model.fit(train_generator,
                         epochs = 15)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.title('Training accuracy')
plt.legend()
fig = plt.figure()
fig.savefig('acc.png')


plt.plot(epochs, loss, 'b', label='Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training loss')

plt.legend()
plt.show()

In [ ]:
model.evaluate(test_generator)

In [ ]:
predict = model.predict(test_generator)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report


y_pred = np.argmax(predict, axis=1)

true_classes = test_generator.classes

class_labels = list(test_generator.class_indices.keys())   

print("CONFUSION MATRIX")
print("================")
print("\n")
print(confusion_matrix(test_generator.classes, y_pred, ))
print("\n")


In [ ]:
print("\CLASSIFICATION REPORT")
print("\n")
report = classification_report(true_classes, y_pred, target_names=class_labels.sort(key=str))
print(report)